# 天池大赛-快来一起挖掘幸福感
#### By 王升
#### 2020.12.24

#### 赛题与数据：https://tianchi.aliyun.com/competition/entrance/231702/information

In [1]:
# 导入库函数
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 1 数据处理及探索

In [2]:
# 读取数据
data_train_raw = pd.read_csv('happiness_train_abbr.csv')
data_test_raw = pd.read_csv('happiness_test_abbr.csv')

# 合并数据
data_train_raw['data_source'] = 1
data_test_raw['data_source'] = 0
data_all = pd.concat([data_train_raw, data_test_raw], axis=0)

In [3]:
# 查看数据大小
print('data_train shape:', data_train_raw.shape)
print('data_test shape:', data_test_raw.shape)
print('data_all shape:', data_all.shape)

data_train shape: (8000, 43)
data_test shape: (2968, 42)
data_all shape: (10968, 43)


In [4]:
# 简要浏览数据
data_all.head()

,birth,car,city,class,county,data_source,depression,edu,equity,family_income,...,status_peer,survey_time,survey_type,view,weight_jin,work_exper,work_manage,work_status,work_type,work_yr
0,1959,2,32,3,59,1,5,11,3,60000.0,...,3,2015/8/4 14:18,1,4,155,1,2.0,3.0,1.0,30.0
1,1992,2,52,6,85,1,3,12,3,40000.0,...,1,2015/7/21 15:04,2,4,110,1,3.0,3.0,1.0,2.0
2,1967,2,83,5,126,1,5,4,4,8000.0,...,2,2015/7/21 13:24,2,4,122,2,NaN,NaN,NaN,NaN
3,1943,1,28,5,51,1,4,3,4,12000.0,...,2,2015/7/25 17:33,2,3,170,4,NaN,NaN,NaN,NaN
4,1994,1,18,1,36,1,3,12,2,-2.0,...,3,2015/8/10 9:50,1,3,110,6,NaN,NaN,NaN,NaN


In [5]:
# 查看数据信息
print(data_train_raw.info(),'\n\n')
print(data_test_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 43 columns):
id                 8000 non-null int64
happiness          8000 non-null int64
survey_type        8000 non-null int64
province           8000 non-null int64
city               8000 non-null int64
county             8000 non-null int64
survey_time        8000 non-null object
gender             8000 non-null int64
birth              8000 non-null int64
nationality        8000 non-null int64
religion           8000 non-null int64
religion_freq      8000 non-null int64
edu                8000 non-null int64
income             8000 non-null int64
political          8000 non-null int64
floor_area         8000 non-null float64
height_cm          8000 non-null int64
weight_jin         8000 non-null int64
health             8000 non-null int64
health_problem     8000 non-null int64
depression         8000 non-null int64
hukou              8000 non-null int64
socialize          8000 non-null

> - **存在空值，需要处理**

In [6]:
# 查看数据基本的统计描述
data_all.describe()

,birth,car,city,class,county,data_source,depression,edu,equity,family_income,...,status_3_before,status_peer,survey_type,view,weight_jin,work_exper,work_manage,work_status,work_type,work_yr
count,10968.000000,10968.000000,10968.000000,10968.000000,10968.000000,10968.000000,10968.000000,10968.000000,10968.000000,1.096700e+04,...,10968.000000,10968.000000,10968.000000,10968.000000,10968.000000,10968.000000,4030.000000,4029.000000,4030.000000,4029.000000
mean,1964.602753,1.817925,42.667123,4.197301,70.787746,0.729395,3.811725,4.857586,3.131838,6.632110e+04,...,1.705689,2.226933,1.410102,3.290026,121.370988,2.984044,2.638710,3.144949,0.908189,14.447009
std,16.897683,0.505590,27.176296,2.027329,38.683432,0.444293,1.087669,3.166881,1.315518,2.835960e+05,...,0.942302,0.960806,0.491874,2.039244,23.150944,1.752649,1.763925,1.756344,1.406120,11.397224
min,1920.000000,-8.000000,1.000000,-8.000000,1.000000,0.000000,-8.000000,-8.000000,-8.000000,-3.000000e+00,...,-8.000000,-8.000000,1.000000,-8.000000,40.000000,1.000000,-8.000000,-8.000000,-8.000000,-3.000000
25%,1952.000000,2.000000,18.000000,3.000000,38.000000,0.000000,3.000000,3.000000,2.000000,1.300000e+04,...,1.000000,2.000000,1.000000,3.000000,105.000000,1.000000,2.000000,3.000000,1.000000,5.000000
50%,1965.000000,2.000000,42.000000,5.000000,73.000000,1.000000,4.000000,4.000000,3.000000,3.840000e+04,...,2.000000,2.000000,1.000000,4.000000,120.000000,3.000000,3.000000,3.000000,1.000000,12.000000
75%,1977.000000,2.000000,65.000000,5.000000,104.000000,1.000000,5.000000,6.000000,4.000000,7.000000e+04,...,2.000000,3.000000,2.000000,4.000000,135.000000,5.000000,3.000000,3.000000,1.000000,22.000000
max,1997.000000,2.000000,89.000000,10.000000,134.000000,1.000000,5.000000,14.000000,5.000000,9.999992e+06,...,3.000000,3.000000,2.000000,5.000000,260.000000,6.000000,4.000000,9.000000,2.000000,55.000000


In [7]:
# 查看标签分布
pd.value_counts(data_train_raw['happiness'], normalize=True)

 4    0.602250
 5    0.176250
 3    0.144875
 2    0.062125
 1    0.013000
-8    0.001500
Name: happiness, dtype: float64

> - **存在无效值 -8，需要剔除**

In [8]:
# 剔除无效target
data_all = data_all[~ ((data_all['data_source'] == 1) & (data_all['happiness'] == -8))]

# 计算年龄
data_all['age'] = pd.to_datetime( data_all['survey_time'], format='%Y/%m/%d %H:%M').dt.year - data_all['birth']

In [9]:
# 逐一查看变量的取值情况
n = 0
col = data_train_raw.columns[n]
print(col, '>>>')
print('train:')
values = pd.concat([data_train_raw[col].value_counts(dropna=False), data_train_raw[col].value_counts(normalize=True, dropna=False)], axis=1)
values.sort_index(inplace=True, na_position='first')
print(values.head())

print('test:')
values = pd.concat([data_test_raw[col].value_counts(dropna=False), data_test_raw[col].value_counts(normalize=True, dropna=False)], axis=1)
values.sort_index(inplace=True, na_position='first')
print(values.head())


# 结合变量解释以及调查问卷问题，处理空值、负值
data_all.loc[(data_all['income'] == -3) | (data_all['income'] == -2) | (data_all['income'] == -1), ['family_m']] = -8
data_all = data_all[~ ((data_all['data_source'] == 1) & (data_all['socialize'] == -8))]
data_all['work_status'].fillna(-8, inplace=True)
data_all['work_yr'].fillna(0, inplace=True)
data_all.loc[(data_all['work_yr'] == -3) | (data_all['work_yr'] == -2) | (data_all['work_yr'] == -1), ['work_yr']] = 0
data_all['work_type'].fillna(-8, inplace=True)
data_all['work_manage'].fillna(-8, inplace=True)
data_all = data_all[~ ((data_all['data_source'] == 1) & (data_all['family_income'].isnull()))]
data_all.loc[(data_all['family_income'] == -3) | (data_all['family_income'] == -2) | (data_all['family_income'] == -1), ['family_income']] = -8
data_all.loc[(data_all['family_m'] == -3) | (data_all['family_m'] == -2) | (data_all['family_m'] == -1), ['family_m']] = -8
data_all.loc[(data_all['house'] == -3) | (data_all['house'] == -2) | (data_all['house'] == -1), ['house']] = -8

id >>>
train:
   id        id
1   1  0.000125
2   1  0.000125
3   1  0.000125
4   1  0.000125
5   1  0.000125
test:
      id        id
8001   1  0.000337
8002   1  0.000337
8003   1  0.000337
8004   1  0.000337
8005   1  0.000337


In [10]:
# 查看每个字段的取值种类数
data_all.nunique()

birth                 78
car                    3
city                  85
class                 11
county               130
data_source            2
depression             6
edu                   15
equity                 6
family_income        444
family_m              14
family_status          6
floor_area           461
gender                 2
happiness              5
health                 6
health_problem         6
height_cm             61
house                 16
hukou                  8
id                 10951
inc_ability            5
income               386
learn                  6
marital                7
nationality            9
political              5
province              28
relax                  6
religion               3
religion_freq         10
socialize              5
status_3_before        4
status_peer            4
survey_time         9521
survey_type            2
view                   6
weight_jin           156
work_exper             6
work_manage            5


In [11]:
# 将变量分为分类变量和连续变量
cols = [col for col in data_all.columns if col not in ['id', 'survey_time', 'birth', 'happiness']]
continuous_variables = ['income', 'floor_area', 'height_cm', 'weight_jin', 'work_yr', 'family_income', 'age']
categorical_variables = [col for col in cols if col not in continuous_variables]

print('cols:\n', cols, '\n')
print('continuous_variables:\n', continuous_variables, '\n')
print('categorical_variables:\n', categorical_variables, '\n')

cols:
 ['car', 'city', 'class', 'county', 'data_source', 'depression', 'edu', 'equity', 'family_income', 'family_m', 'family_status', 'floor_area', 'gender', 'health', 'health_problem', 'height_cm', 'house', 'hukou', 'inc_ability', 'income', 'learn', 'marital', 'nationality', 'political', 'province', 'relax', 'religion', 'religion_freq', 'socialize', 'status_3_before', 'status_peer', 'survey_type', 'view', 'weight_jin', 'work_exper', 'work_manage', 'work_status', 'work_type', 'work_yr', 'age'] 

continuous_variables:
 ['income', 'floor_area', 'height_cm', 'weight_jin', 'work_yr', 'family_income', 'age'] 

categorical_variables:
 ['car', 'city', 'class', 'county', 'data_source', 'depression', 'edu', 'equity', 'family_m', 'family_status', 'gender', 'health', 'health_problem', 'house', 'hukou', 'inc_ability', 'learn', 'marital', 'nationality', 'political', 'province', 'relax', 'religion', 'religion_freq', 'socialize', 'status_3_before', 'status_peer', 'survey_type', 'view', 'work_exper', 

In [12]:
# 结合变量解释将分类变量分为无序分类和有序分类2种
non_ordered_categorical_variables = ['survey_type', 'province', 'city', 'county', 'gender', 'nationality', 'religion', 'hukou', 'work_exper', 'work_status', 'work_type', 'work_manage', 'marital']
ordered_categorical_variables = [col for col in categorical_variables if col not in non_ordered_categorical_variables]

print('non_ordered_categorical_variables:\n', non_ordered_categorical_variables)
print('ordered_categorical_variables:\n', ordered_categorical_variables)

non_ordered_categorical_variables:
 ['survey_type', 'province', 'city', 'county', 'gender', 'nationality', 'religion', 'hukou', 'work_exper', 'work_status', 'work_type', 'work_manage', 'marital']
ordered_categorical_variables:
 ['car', 'class', 'data_source', 'depression', 'edu', 'equity', 'family_m', 'family_status', 'health', 'health_problem', 'house', 'inc_ability', 'learn', 'political', 'relax', 'religion_freq', 'socialize', 'status_3_before', 'status_peer', 'view']


In [13]:
data_all[cols] = data_all[cols].astype('int64')

In [14]:
# 对无序分类变量做 one_hot 编码
non_ordered_dummies = pd.get_dummies(data_all[non_ordered_categorical_variables], columns=non_ordered_categorical_variables, dtype='int32')
data_all.drop(columns=non_ordered_categorical_variables, axis=1, inplace=True)
data_all = pd.concat([data_all, non_ordered_dummies], axis=1)
data_all.head()

,birth,car,class,data_source,depression,edu,equity,family_income,family_m,family_status,...,work_manage_2,work_manage_3,work_manage_4,marital_1,marital_2,marital_3,marital_4,marital_5,marital_6,marital_7
0,1959,2,3,1,5,11,3,60000,2,2,...,1,0,0,0,0,1,0,0,0,0
1,1992,2,6,1,3,12,3,40000,3,4,...,0,1,0,1,0,0,0,0,0,0
2,1967,2,5,1,5,4,4,8000,3,3,...,0,0,0,0,0,1,0,0,0,0
3,1943,1,5,1,4,3,4,12000,3,3,...,0,0,0,0,0,0,0,0,0,1
4,1994,1,1,1,3,12,2,-8,-8,3,...,0,0,0,1,0,0,0,0,0,0


In [15]:
# 重新将数据拆为 train 和 test
data_train = data_all.loc[(data_all['data_source'] == 1), :].drop(columns=['data_source'], axis=1)
data_test = data_all.loc[(data_all['data_source'] == 0), :].drop(columns=['data_source'], axis=1)

In [16]:
# 将数据分为X和y
X = data_train[[col for col in data_train.columns if col not in ['id', 'survey_time', 'birth', 'happiness']]]
y = data_train['happiness']

## 2 模型训练

In [17]:
# 导入模型工具包
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [18]:
# 连续变量标准化
X[continuous_variables] = StandardScaler().fit_transform(X[continuous_variables])

In [19]:
# 拆分训练集和测试集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
# lgb
model_lgb = LGBMRegressor(num_leaves=2**4, learning_rate=0.05, class_weight='balanced', n_estimators=400, random_state=42, metric='l2')
model_lgb.fit(X_train, y_train)

y_train_pred_lgb = model_lgb.predict(X_train)
MSE_lgb = mse(y_train, y_train_pred_lgb)
print('训练集MSE_lgb：', MSE_lgb)

y_val_pred_lgb = model_lgb.predict(X_val)
MSE_lgb = mse(y_val, y_val_pred_lgb)
print('验证集MSE_lgb：', MSE_lgb)

训练集MSE_lgb： 0.33923744027686076
验证集MSE_lgb： 0.5895474323791815


In [21]:
# xgboost
model_xgb = XGBRegressor(n_estimators=400, max_depth=4, learning_rate=0.05, random_state=42)
model_xgb.fit(X_train, y_train)

y_train_pred_xgb = model_xgb.predict(X_train)
MSE_xgb = mse(y_train, y_train_pred_xgb)
print('训练集MSE_xgb：', MSE_xgb)

y_val_pred_xgb = model_xgb.predict(X_val)
MSE_xgb = mse(y_val, y_val_pred_xgb)
print('验证集MSE_xgb：', MSE_xgb)

训练集MSE_xgb： 0.3111609332338382
验证集MSE_xgb： 0.461504863622938


In [22]:
# 随机森林
model_rf = RandomForestRegressor(n_estimators=400, max_depth=5, n_jobs=-1, random_state=42)
model_rf.fit(X_train, y_train)

y_train_pred_rf = model_rf.predict(X_train)
MSE_rf = mse(y_train, y_train_pred_rf)
print('训练集MSE_rf：', MSE_rf)


y_val_pred_rf = model_rf.predict(X_val)
MSE_rf = mse(y_val, y_val_pred_rf)
print('验证集MSE_rf：', MSE_rf)

训练集MSE_rf： 0.45747621224021817
验证集MSE_rf： 0.4985322181603867


In [23]:
# 模型融合
y_val_pred_weighted = (1/MSE_lgb * y_val_pred_lgb + 1/MSE_xgb * y_val_pred_xgb + 1/MSE_rf * y_val_pred_rf)/(1/MSE_lgb + 1/MSE_xgb + 1/MSE_rf)
MSE_weighted = mse(y_val, y_val_pred_weighted)
print('MSE_weighted：', MSE_weighted)

MSE_weighted： 0.46538094679784275


In [24]:
# 使用全部数据重新训练
model_lgb.fit(X, y)
y_pred_lgb = model_lgb.predict(X)
MSE_lgb = mse(y, y_pred_lgb)

model_xgb.fit(X, y)
y_pred_xgb = model_xgb.predict(X)
MSE_xgb = mse(y, y_pred_xgb)

model_rf.fit(X, y)
y_pred_rf = model_rf.predict(X)
MSE_rf = mse(y, y_pred_rf)


# 模型融合
y_pred_weighted = (1/MSE_lgb * y_pred_lgb + 1/MSE_xgb * y_pred_xgb + 1/MSE_rf * y_pred_rf)/(1/MSE_lgb + 1/MSE_xgb + 1/MSE_rf)
MSE_weighted = mse(y, y_pred_weighted)
print('MSE_weighted：', MSE_weighted)

MSE_weighted： 0.33720458378365


## 3 预测

In [25]:
# 剔除列
X_test = data_test[[col for col in data_test.columns if col not in ['id', 'survey_time', 'birth', 'happiness']]]

# 连续变量标准化
X_test[continuous_variables] = StandardScaler().fit_transform(X_test[continuous_variables])

In [26]:
# 预测
y_test_pred_lgb = model_lgb.predict(X_test)
y_test_pred_xgb = model_xgb.predict(X_test)
y_test_pred_rf = model_rf.predict(X_test)


# 模型融合
y_test_pred_weighted = (1/MSE_lgb * y_test_pred_lgb + 1/MSE_xgb * y_test_pred_xgb + 1/MSE_rf * y_test_pred_rf)/(1/MSE_lgb + 1/MSE_xgb + 1/MSE_rf)

In [27]:
# 预测结果
result = pd.DataFrame()
result['id'] = data_test['id']
result['happiness'] = np.round(y_test_pred_weighted,4)
result.loc[result['happiness'] < 1, ['happiness']] = 1
result.loc[result['happiness'] > 5, ['happiness']] = 5
result.head()

,id,happiness
0,8001,3.5514
1,8002,2.7407
2,8003,3.1143
3,8004,4.4008
4,8005,3.3705


In [28]:
# 导出预测结果
result.to_csv('test_happiness_submit.csv', index=False)

### 总结：
- 数据预处理：虽然耗时耗力且枯燥乏味，但数据预处理很重要，需要逐个变量去观察并做对应处理
- 模型选择：经过迭代测试发现，基于集成算法的模型比较好，对 xgb、lgb、rf 做模型融合能得到更好的结果
- 模型调参：要同时关注训练集和验证集的评分，避免过拟合，可以选择手动调整参数并观察结果，也可以选择使用网格搜索